In [1]:
print("Hello LangChain")

Hello LangChain


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY[:10])
print(OPENAI_API_KEY[:5])

gsk_3


In [20]:
# prompt + llm + output 

# prompt: (쉽게 말해) AI에게 하는 질문
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , # AI에게 역할을 주는 것.
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [30]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [ ]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq는 ChatGPT와 호환 -> ChatGPT 모델 사용 가능
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성 (모델 가져와서 생성)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct", # 사용 모델: llama-4-scout
    #model="moonshotai/kimi-k2-instruct-0905", # 사용 모델: kimi-k2
    model="openai/gpt-oss-120b", # 사용 모델: gpt-oss-120b
    temperature=0.7
)
print(type(llm)) # 타입: ChatOpenAI
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x1402c5990> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1402c6bd0> root_client=<openai.OpenAI object at 0x1402c41d0> root_async_client=<openai.AsyncOpenAI object at 0x1402c68d0> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [45]:
# 생성한 모델에 질문하기(prompt 입력)
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
응답: ## 파이썬(Python)이란?

**파이썬**은 1991년 네덜란드의 프로그래머 **귀도 반 로썸(Guido van Rossum)**이 발표한 고수준(high‑level), 인터프리터 방식(interpreted) 프로그래밍 언어입니다. “파이썬”이라는 이름은 로썸이 영국 코미디 그룹 *Monty Python*의 팬이었기 때문에 붙여졌으며, 그 유머 감각이 언어 설계 철학에도 반영되었습니다.  

파이썬은 **읽기 쉬운 문법**, **폭넓은 표준 라이브러리**, **다양한 분야에서의 활용성**을 바탕으로 전 세계 개발자·연구자·학생들에게 사랑받고 있습니다.

---

## 1. 파이썬의 핵심 특징

| 특징 | 설명 | 장점 |
|------|------|------|
| **읽기 쉬운 문법** | 들여쓰기(Indentation)로 블록을 구분하고, 불필요한 구문(세미콜론, 중괄호 등)이 최소화됨 | 코드 가독성·유지보수성 향상 |
| **동적 타이핑(Dynamic Typing)** | 변수에 타입을 미리 선언하지 않아도 런타임에 자동으로 결정 | 빠른 프로토타이핑, 생산성 ↑ |
| **인터프리터 방식** | 소스 코드를 바로 실행(컴파일 단계가 내부적으로 자동) | 디버깅·실험이 쉬움 |
| **풍부한 표준 라이브러리** | 파일 I/O, 네트워킹, 웹, 데이터베이스, 암호화 등 2000여 개 모듈 제공 | 외부 패키지 없이도 다양한 작업 가능 |
| **멀티패러다임 지원** | 절차적, 객체지향, 함수형 프로그래밍 모두 지원 | 상황에 맞는 스타일 선택 가능 |
| **플랫폼 독립성** | Windows, macOS, Linux, iOS, Android 등 거의 모든 OS에서 실행 | “Write once, run anywhere” |
| **대규모 생태계** | PyPI(Python Package Index)에 400,000개 이상의 서드파티 패키지 | 데이터 과

# LCEL

In [41]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [48]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1402c5990>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1402c6bd0>, root_client=<openai.OpenAI object at 0x1402c41d0>, root_async_client=<openai.AsyncOpenAI object at 0x1402c68d0>, model_name='openai/gpt-oss-120b', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputParser()


In [51]:
response = chain.invoke({"input":"LangChain이 무엇인가요?"})

In [52]:
print(type(response))
print(response)

<class 'str'>
## LangChain이란?

**LangChain**은 **대형 언어 모델(LLM)** 을 활용한 **애플리케이션**을 보다 **쉽고 구조화된 방식**으로 만들 수 있게 해 주는 **오픈소스 프레임워크**입니다.  
주로 **Python**과 **JavaScript/TypeScript**에서 사용되며, LLM을 단순히 “텍스트를 생성”하는 수준을 넘어 **다양한 외부 도구·데이터·워크플로와 연결**하고, **복잡한 로직을 구성**할 수 있게 도와줍니다.

---

## 핵심 개념

| 개념 | 설명 | 예시 |
|------|------|------|
| **Chain** | 여러 LLM 호출·프롬프트·전처리·후처리 단계를 순차·분기·반복적으로 연결한 흐름 | “사용자 질문 → 검색 → 문서 요약 → 답변 생성” |
| **PromptTemplate** | 변수(플레이스홀더)를 포함한 프롬프트 템플릿을 정의하고, 실행 시 값 삽입 | `template = "다음 문서를 요약해줘: {text}"` |
| **LLM** | OpenAI, Anthropic, Cohere, Azure, HuggingFace 등 다양한 모델에 대한 래퍼 | `ChatOpenAI(model="gpt-4o")` |
| **Memory** | 대화/작업 이력을 저장해 다음 호출에 활용 (컨텍스트 유지) | 대화형 챗봇에서 이전 질문·답변을 기억 |
| **Agents** | “도구를 사용할 수 있는 LLM” → LLM이 판단해 검색, 계산, 데이터베이스 질의 등 외부 도구를 호출 | “날씨를 알려줘” → LLM이 날씨 API 호출 후 결과 반환 |
| **Retrievers** | 벡터스토어·검색엔진 등에서 관련 문서를 찾아 반환 (RAG: Retrieval‑Augmented Generation) | `FAISS`, `Pinecone`, `ElasticSearch` 등 |
| **Callbacks / Tracing** | 실행 흐름을 로깅·시각화해 디버깅·모니터링